# The Y2038 Problem

**อ้างอิง:**

- [CS50 2021 in HDR - Lecture 1 - C](https://youtu.be/URrzmoIyqLw?t=8485)
- [gnu.org, Time Types](https://www.gnu.org/software/libc/manual/html_node/Time-Types.html)
- [cppreference.com, time_t](https://en.cppreference.com/w/c/chrono/time_t)

## TL;DR

- คอมพิวเตอร์ระบบ UNIX มีวิธีในการ track เวลาด้วยการนับ `bit วินาที` เพิ่มขึ้นเรื่อย ๆ จากจุด ๆ หนึ่ง
- จุดเริ่มต้นของการนับเวลาคือ `UNIX Epoch` – มีค่าวินาทีเป็น `0` และเทียบได้เป็นวันที่ 1970-01-01, 00:00:00 (UTC)
- คอมพิวเตอร์เก็บเวลาที่ elapsed จาก `UNIX Epoch` ในรูป `signed 32-bit integer` นั่นหมายความว่าเราจะนับเวลาได้สูงสุด<br>
$2^{31} - 1 = 2147483647$ วินาทีจากจุด epoch
- เมื่อวินาทีถูกนับเพิ่มเป็น $2^{31}$ เลยเกิด `integer overflow`

----

## The UNIX Epoch

UNIX Epoch คือจุดนับเวลาที่มี bit เริ่มต้นเป็น `00000000000000000000000000000000` (signed) ก็คือ `0` อะแหละ

ลองแปลงเวลา ณ `UNIX Epoch`

- `time_t` คือ datatype หนึ่งที่ใช้ represent เวลาตามปฏิทิน

In [1]:
#include <stdio.h>
#include <time.h>
 
int main(void)
{
    // ให้ตัวแปร epoch แทนจุดที่ตัวนับเวลามีค่าเท่ากับ 0
    time_t epoch = 0;
    
    // print เวลาตามปฏิทินที่แปลงมาจากค่าของตัวแปร `epoch` ซึ่งมีค่าเท่ากับ 0 ใน time_t datatype
    printf("%s", asctime(gmtime(&epoch)));
    
    return 0;
}

Thu Jan  1 00:00:00 1970


**ทีนี้ลองแปลงเวลาที่ `epoch + 1`**

In [2]:
#include <stdio.h>
#include <time.h>
 
int main(void)
{
    // ให้ตัวแปร epoch แทนจุดที่ตัวนับเวลามีค่าเท่ากับ 0
    time_t epoch = 0;
    
    // ให้ตัวแปร timestamp แทนจุดที่ตัวนับเวลามีค่าเท่ากับ epoch + 1
    time_t timestamp = epoch + 1;
    
    // print เวลาตามปฏิทินที่แปลงมาจากค่าของตัวแปร `timestamp` ซึ่งมีค่าเท่ากับ epoch + 1 ใน time_t datatype
    printf("%s", asctime(gmtime(&timestamp)));
    
    return 0;
}

Thu Jan  1 00:00:01 1970


**จะเห็นว่าเวลาเปลี่ยนจาก `00:00:00` เป็น `00:00:01` แต่ยังอยู่ในวันที่เดียวกัน**

## ทีนี้ลองแปลงเวลาที่ $2^{32} - 1$

$2^{32} - 1 = 4294967295$

In [3]:
#include <stdio.h>
#include <time.h>
 
int main(void)
{
    time_t epoch = 0;
    time_t timestamp = epoch + 2147483647;
    
    printf("%s", asctime(gmtime(&timestamp)));
    
    return 0;
}

Tue Jan 19 03:14:07 2038


## ลอง `+= 1` ใน 32-bit program

**เนื่องจากเครื่องของผมมี architecture แบบ 64-bit เลยต้องบังคับให้ compiler compile เป็นแบบ 32-bit**

In [7]:
#include <stdio.h>
#include <time.h>
 
int main(void)
{
    time_t epoch = 0;
    time_t timestamp = epoch + 2147483648;
    
    printf("%s", asctime(gmtime(&timestamp)));
    
    return 0;
}

Tue Jan 19 03:14:08 2038
